In [ ]:
def yolo2xyxy(bboxes,H,W):
    ''' bboxes is an array of shape (N,4) for x_c, y_c, w,h in yolo format'''
    xmin = (bboxes[:,0] - bboxes[:,2]/2)*W
    ymin = (bboxes[:,1] - bboxes[:,3]/2)*H
    xmax = (bboxes[:,0] + bboxes[:,2]/2)*W
    ymax = (bboxes[:,1] + bboxes[:,3]/2)*H

    return np.stack([xmin,ymin,xmax,ymax], axis=1)

def GT_yolo2xyxy(src_dir, dest_dir, to_run = False):
    if to_run:
        os.makedirs(dest_dir, exist_ok=True)
        txt_pths = natsorted(get_files(src_dir, extensions='.txt'))
        num_txts = len(txt_pths); print(f'converting {num_txts} files to xyxy format')
        for txt_pth in txt_pths:
            name = txt_pth.name
            im_pth = src_dir+'/'+name.replace('.txt','.jpg')
            im = cv2.imread(im_pth)
            H,W = im.shape[:2]
            with open(txt_pth, 'r') as f:
                #print('GT parsing: ',name)
                lines = f.readlines()
                f.close()

            ls = [list(map(float,l.split())) for l in lines] # each line as a list of floats
            arr = np.stack(ls) # convert to array for easy indexing
            class_labels = arr[:,0]
            bboxes = arr[:,1:]
            xyxy_boxes = yolo2xyxy(bboxes,H,W)

            new_txt_pth = dest_dir+'/'+name
            with open(f'{new_txt_pth}', 'w+') as txt_file: 
                for class_id, box in zip(class_labels, xyxy_boxes):
                    class_id = np.array([class_id], dtype=int)
                    box = np.array(box, dtype=int)

                    l = np.concatenate([class_id, box]).tolist()
                    yolo_line = ' '.join(map(str, l)) + '\n'
                    txt_file.write(yolo_line)  
                txt_file.close()
        print('******* DONE yolo2xyxy*******')        
    else:
        print('yolo to xyxy not running')

            
def preds_yolo2xyxy(src_dir, dest_dir, to_run = False):
    if to_run:
        os.makedirs(dest_dir, exist_ok=True)
        txt_pths = natsorted(get_files(src_dir, extensions='.txt'))
        num_txts = len(txt_pths); print(f'converting {num_txts} files to xyxy format')
        for txt_pth in txt_pths:
            name = txt_pth.name
            im_pth = src_dir+'/'+name.replace('.txt','.jpg')
            im = cv2.imread(im_pth)
            H,W = im.shape[:2]
            with open(txt_pth, 'r') as f:
                #print('preds parsing: ',name)
                lines = f.readlines()
                f.close()

            ls = [list(map(float,l.split())) for l in lines] # each line as a list of floats
            arr = np.stack(ls) # convert to array for easy indexing
            class_labels = arr[:,0]
            scores = arr[:,1];#ones_array = np.ones_like(scores)
            bboxes = arr[:,2:]
            xyxy_boxes = yolo2xyxy(bboxes,H,W)

            new_txt_pth = dest_dir+'/'+name
            with open(f'{new_txt_pth}', 'w+') as txt_file: 
                for class_id, conf_score, box in zip(class_labels, scores, xyxy_boxes):
                    class_id = int(class_id)  # Ensure class_id is an integer
                    conf_score = float(conf_score)  # Ensure confidence score is a float
                    box = box.astype(int)  # Ensure bounding box coordinates are integers
                    l = [class_id, conf_score] + box.tolist()
                    yolo_line = ' '.join(map(str, l)) + '\n'
                    txt_file.write(yolo_line)
                txt_file.close()
        print('******* DONE yolo2xyxy*******')
    else:
        print('yolo to xyxy not running')